In [1]:
from pathlib import Path

import autochem as ac
import automech
import automol
import more_itertools as mit
import numpy as np
from project_utilities import p_
from protomech import mess

T_range = (300, 1200)
T_vals = (600, 700, 800, 900, 1000, 1100, 1200)
P_vals = (0.1, 1, 10, 100)
T_drop = (300, 400)  # Drop low-T rates for fit
A_fill = 1e-20
compare = False

stoich = "C5H7"
clear_nodes = []
clear_edges = []

# stoich = "C5H7O2"
# clear_nodes = []  # was clearing: ["C5H6O(645)z+OH(4)", "CPND2(626)+OH(4)"]
# clear_edges = []

# stoich = "C5H9"
# clear_nodes = []
# clear_edges = []

# stoich = "C5H9O"
# clear_nodes = []
# clear_edges = []

# stoich = "C5H9O2"
# clear_nodes = []
# clear_edges = [
#     # Well-skipping:
#     ("C5H8O(833)+OH(4)", "C5H8O(829)rs+OH(4)")
# ]

# stoich = "C5H11O"  # suspicious -- doesn't seem to compare well with theory
# clear_nodes = []
# clear_edges = []

# stoich = "C5H11O2"
# clear_nodes = []
# clear_edges = []

tag = "Z_mess_v0"
root_path = Path("../..")
stoich_path = Path(stoich)

In [2]:
mech = automech.io.read(p_.stereo_mechanism(tag, "json", p_.data(root_path)))
mech.reactions = automech.reaction.select_pes(mech.reactions, stoich)
mech = automech.without_unused_species(mech)

In [3]:
mess_inp = stoich_path / "mess.inp"
surf = mess.surf.from_mess_input(mess_inp)
mess.net.display(surf, mech=mech, height="1000px")

In [4]:
from importlib import reload
from protomech.mess import surf as m_surf_

reload(m_surf_)

mess_out = stoich_path / "rate.out"
surf = m_surf_.with_mess_output_rates(surf, mess_out=mess_out)
surf = m_surf_.absorb_fake_nodes(surf)
m_surf_.forward_rate_keys(surf, mech, direct=True, well_skipping=False)
m_surf_.forward_rate_keys(surf, mech, direct=False, well_skipping=True)

[(4, 6),
 (0, 6),
 (6, 2),
 (0, 3),
 (3, 6),
 (5, 2),
 (0, 4),
 (5, 0),
 (4, 2),
 (5, 3)]